In [22]:
import numpy as np
import pandas as pd
import time
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score


In [23]:
file_path = "dataset/wine.data"

columns = [
    "Class", "Alcohol", "Malic acid", "Ash", "Alcalinity of ash", "Magnesium",
    "Total phenols", "Flavanoids", "Nonflavanoid phenols", "Proanthocyanins",
    "Color intensity", "Hue", "OD280/OD315", "Proline"
]

# Загружаем датасет
df = pd.read_csv(file_path, header=None, names=columns)


X = df.iloc[:, 1:] 
y = df.iloc[:, 0]   



In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Параметры для GridSearch
# Param_grid в контексте библиотеки scikit-learn — словарь, задающий сетку параметров для поиска.
# Этот параметр используется в инструменте GridSearchCV,
# который позволяет выполнять исчерпывающий поиск по
# заданным значениям параметров для оценщика.
param_grid = {
    'DecisionTree': {'classifier__max_depth': [3, 5,]},
    'RandomForest': {'classifier__n_estimators': [30, 50], 'classifier__max_depth': [5, 10]},
    'NaiveBayes': {'classifier__var_smoothing': [1e-4]},
    'SVM': {'classifier__C': [0.1, 0,8, 10], 'classifier__kernel': ['linear', 'rbf']}
}


In [25]:

# Модели и пайплайны

# Модели и пайплайны с настройкой гиперпараметров
models2 = {
    'DecisionTree': Pipeline([
        ('scaler', StandardScaler()), 
        ('classifier', DecisionTreeClassifier(max_depth=5, min_samples_split=10, criterion='entropy'))
    ]),
    'RandomForest': Pipeline([
        ('scaler', StandardScaler()), 
        ('classifier', RandomForestClassifier(n_estimators=100, max_depth=10, bootstrap=True, random_state=42))
    ]),
    'NaiveBayes': Pipeline([
        ('scaler', StandardScaler()), 
        ('classifier', GaussianNB(var_smoothing=1e-9))
    ]),
    'SVM': Pipeline([
        ('scaler', StandardScaler()), 
        ('classifier', SVC(C=1.0, kernel='rbf', gamma='scale'))
    ])
}



In [26]:
# Сравнение методов в исходном пространстве
results = []
for model_name, model in models2.items():
    start = time.time()
    
    model.fit(X_train, y_train)  # Обучаем модель
    train_time = time.time() - start

    start = time.time()
    y_pred = model.predict(X_test)  # Делаем предсказания
    predict_time = time.time() - start

    # Сохраняем результаты
    results.append({
        'Model': model_name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Train Time': train_time,
        'Predict Time': predict_time
    })


In [27]:
# Сравнение с разной размерностью через PCA
dims = [2, 5, 10]  # Количество компонент PCA
pca_results = []

for dim in dims:
    # Применяем PCA к стандартизированным данным
    pca = PCA(n_components=dim)
    X_pca = pca.fit_transform(StandardScaler().fit_transform(X))
    
    # Разделение на обучающую и тестовую выборку
    X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca, y, test_size=0.3, random_state=42)

    for model_name, model in models2.items():
        start = time.time()
        model.fit(X_train_pca, y_train_pca)  # Обучение модели
        train_time = time.time() - start

        start = time.time()
        y_pred = model.predict(X_test_pca)  # Предсказание
        predict_time = time.time() - start

        pca_results.append({
            'Dimension': dim,
            'Model': model_name,
            'Accuracy': accuracy_score(y_test_pca, y_pred),
            'Train Time': train_time,
            'Predict Time': predict_time
        })

# Выводим результаты
import pandas as pd
pca_results_df = pd.DataFrame(pca_results)
print(pca_results_df)


# Вывод результатов
pd.set_option('display.max_columns', None)    # Снимаем ограничения с вывода
pd.set_option('display.max_colwidth', None)   # Снимаем ограничения с вывода

print("Результаты в исходном пространстве:")
print(pd.DataFrame(results))

print("\nРезультаты с PCA:")
print(pd.DataFrame(pca_results))



    Dimension         Model  Accuracy  Train Time  Predict Time
0           2  DecisionTree  0.981481    0.001723      0.001528
1           2  RandomForest  0.981481    0.071614      0.008312
2           2    NaiveBayes  0.981481    0.005330      0.000000
3           2           SVM  0.981481    0.002988      0.000000
4           5  DecisionTree  0.962963    0.000000      0.000000
5           5  RandomForest  0.962963    0.075325      0.000000
6           5    NaiveBayes  0.981481    0.010293      0.000000
7           5           SVM  1.000000    0.000000      0.000000
8          10  DecisionTree  0.962963    0.000713      0.000946
9          10  RandomForest  0.962963    0.073516      0.000000
10         10    NaiveBayes  1.000000    0.000000      0.000000
11         10           SVM  0.962963    0.007335      0.000000
Результаты в исходном пространстве:
          Model  Accuracy  Train Time  Predict Time
0  DecisionTree  0.796296    0.009859      0.000000
1  RandomForest  1.000000   

In [28]:
# hyperparam_results_df = pd.DataFrame(hyperparam_results).T
# dims = [10,20,30]
# pca_results = []
# for dim in dims:
#     pca = PCA(n_components=dim)
#     X_pca = pca.fit_transform(StandardScaler().fit_transform(X))
#     X_train_pca, X_test_pca, y_train_pca, y_test_pca = \
#     train_test_split(X_pca, y, test_size = 0.3, random_state = 42)

#     for model_name in models:
#         model = models[model_name]
#         params = param_grid[model_name]
#         grid = GridSearchCV(model, params, cv = 10)

#         start = time.time()
#         grid.fit(X_train_pca, y_train_pca)
#         train_time = time.time() - start

#         start = time.time()
#         y_pred = grid.predict(X_test_pca)
#         predict_time = time.time() - start

#         pca_results.append({
#             'Размерность': dim,
#             'Метод': model_name,
#             'Точность': accuracy_score(y_test_pca, y_pred),
#             'Время': train_time,
#             'Предикт': predict_time,
#         })